In [2]:
import os
from botocore.config import Config
import boto3

from dotenv import load_dotenv
from openai import OpenAI

# Initialize client to query Qwen2.5 7B Instruct on Predibase using the OpenAI client.

_ = load_dotenv()

# client = OpenAI(
#     base_url=os.environ["PREDIBASE_MODEL_QWEN_URL"], # Qwen 2.5 7B Instruct
#     api_key=os.environ["PREDIBASE_API_KEY"],
# )

BEDROCK_RUNTIME = "bedrock-runtime"
CRIS_CLAUDE_SONNET_37 = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

cfg = Config(
            region_name="us-east-1",
            connect_timeout=10,  # handshake timeout
            read_timeout=3600,  # wait up to 60 min for long generations
            retries={"max_attempts": 3, "mode": "standard"},
        )

bedrock_client = boto3.client(BEDROCK_RUNTIME, config=cfg)

In [3]:
from transformers import AutoTokenizer

base_model_id = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
SYSTEM_PROMPT = """
You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close
your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to
first add your step by step thought process within <think> </think>
tags. Then, return your guessed word in the following format:
<guess> guessed-word </guess>.
"""

In [5]:
from dataclasses import dataclass
from enum import Enum
from typing import List


class LetterFeedback(Enum):
    CORRECT = "✓"
    WRONG_POS = "-"
    WRONG_LETTER = "x"


@dataclass
class GuessWithFeedback:
    guess: str
    feedback: List[LetterFeedback]

    def __repr__(self) -> str:
        """Returns a readable string showing the guess alongside
        its letter-by-letter feedback."""
        feedback_str = " ".join(f"{letter}({fb.value})" for letter, fb in zip(self.guess, self.feedback))
        return f"{self.guess} → Feedback: {feedback_str}"

In [6]:
def render_user_prompt(past_guesses: List[GuessWithFeedback]) -> str:
    """Creates a user-facing prompt that includes past guesses
    and their feedback."""
    prompt = "Make a new 5-letter word guess."
    if past_guesses:
        prompt += "\n\nHere is some previous feedback:"
        for i, guess in enumerate(past_guesses):
            prompt += f"\nGuess {i+1}: {guess}"
    return prompt

In [7]:
def render_prompt(past_guesses: List[GuessWithFeedback]):
    """Formats a full chat prompt using a system message, user
    prompt, and assistant preamble to start the model's
    step-by-step reasoning."""
    messages = [
        {
            "role": "system",
            "content": { "text": SYSTEM_PROMPT }
        },
        {
            "role": "user",
            "content": { "text": render_user_prompt(past_guesses) }
        },
    ]

    return tokenizer.apply_chat_template(
        messages, tokenize=False, continue_final_message=True
    )

In [9]:
def generate_stream(prompt: str, adapter_id: str = "") -> str:
    """Streams a model-generated response from a prompt in
    real-time and prints it as it arrives."""
    body = {
            "messages": prompt,
            "inferenceConfig": {
                "temperature": 0.1,
                "maxTokens": 31000,
            }
        }
    response = bedrock_client.converse(
                modelId=CRIS_CLAUDE_SONNET_37,
                **body
            )
    output = response.get("output", {})
    message = output.get("message", {})
    return message

In [10]:
secret_word = "CRAFT"

past_guesses = [
    GuessWithFeedback(
        "CRANE", [
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.WRONG_LETTER,
            LetterFeedback.WRONG_LETTER,
        ]),
    GuessWithFeedback(
        "CRASH", [
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.WRONG_LETTER,
            LetterFeedback.WRONG_LETTER,
        ]),
]

prompt = render_prompt(past_guesses)
print(prompt)

<|im_start|>system

You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close
your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to
first add your step by step thought process within <think> </think>
tags. Then, return your guessed word in the following format:
<guess> guessed-word </guess>.
<|im_end|>
<|im_start|>user
Make a n

In [11]:
base_completion = generate_stream(prompt)

TypeError: generate_stream() missing 1 required positional argument: 'messages'